<a href="https://colab.research.google.com/github/leihuang/scrapbook/blob/master/decorrelation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import division, print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [0]:
n = 1000  # sample size
rho = 0.5  # correlation
p = 3  # total number of features
k = 1  # the first k features are regressed first and decorrelated from
sigma = 0.1  # noise

mu = [0] * p
covarmat = np.identity(p)*(1-rho) + np.ones((p,p))*rho

np.random.seed(0)
X = np.random.multivariate_normal(mu, covarmat, size=n)

b = np.array([1, 2, 3])  # the beta vector
y = np.dot(X, b) + np.random.randn(n)*sigma

Given an $n$-by-$p$ matrix $X$  ($n>p$), a **projection operator** that projects any $y \in \mathbb{R}^n$ onto the subspace spanned by (the column vectors of) $X$ can be represented by an $n$-by-$n$ matrix:

$H = X(X^T X)^{-1} X^T$ (aka the "*hat matrix*" in linear regression literature). 

It's easy to verify: 
* $HX = X(X^T X)^{-1} X^T X = X$
* $HH = X(X^T X)^{-1} X^T X(X^T X)^{-1} X^T = X(X^T X)^{-1} X^T = H$

Both are consistent with that $H$ represents a projection operator onto the $X$ subspace.

The projection operator to the orthogonal complement of $X$ can be represented by $I - H$. 

* $\langle H y, (I-H) y \rangle = y^T H^T (I - H) y = y^T H^T y - y^T H^T H y = y^T H^T y - y^T H H y = 0$ (it makes use of $H^T = (X(X^T X)^{-1} X^T)^T = X (X^T X)^{-1} X^T = H$ and that the inverse of a symmetric matrix is [also symmetric](https://math.stackexchange.com/a/3162436/224349)).

---

Computing $X(X^T X)^{-1} X^T$: 

SVD: $X = U \Sigma V^T$ 

$X(X^T X)^{-1} X^T = U \Sigma V^T (V \Sigma U^T U \Sigma V^T)^{-1} V \Sigma U^T = U \Sigma V^T (V \Sigma^2 V^T)^{-1} V \Sigma U^T = U \Sigma V^T V \Sigma^{-2} V^T V \Sigma U^T = U U^T$

In [0]:
def decorrelate(X, k):
    """Return the last p-k columns of X (that is, X[:,k:]) decorrelated 
    from the first k columns (that is, X[:,:k]). 
    """
    assert k < X.shape[1]
    U, S, Vt = np.linalg.svd(X[:,:k], full_matrices=False)
    return X[:,k:] - np.dot(np.dot(U, U.T), X[:,k:])

In [4]:
# regress on all features and recover the coefficients

f = LinearRegression(fit_intercept=False).fit(X, y)
f.coef_

array([0.99951928, 1.99832752, 2.99811068])

In [5]:
# regress on the first k features and get biased coefficients

X1, X2_ = X[:,:k], decorrelate(X, k)

f1 = LinearRegression(fit_intercept=False).fit(X1, y)
f1.coef_

array([3.32317414])

In [6]:
# regress on the other n-k features and get biased coefficients

f2 = LinearRegression(fit_intercept=False).fit(X[:,k:], y)
f2.coef_

array([2.3276821 , 3.31870148])

In [7]:
# decorrelate the other n-k features and regress on them,
# and recover the coefficients

f2_ = LinearRegression(fit_intercept=False).fit(X2_, y)
f2_.coef_

array([1.99832752, 2.99811068])

In [8]:
# the decorrelated predictions match the true ones

np.abs(f.predict(X) - (f1.predict(X1) + f2_.predict(X2_))).max()

7.105427357601002e-15